In [1]:
import nltk
import numpy
import pandas
import sklearn
import sklearn.pipeline
import sklearn.grid_search
import sklearn.decomposition

# all 3 lines below should be uncommented
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

import sqlite3

import csshelper
import cssfeature
import csspipe
import csstransformer


db_path = './data.sqlite3'
sqlite_connection = sqlite3.connect(db_path)

/usr/local/lib/python3.4/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
data_reader_collection = [
        ('cExt', csshelper.CExtReader(sqlite_connection))
    
        , ('cNeu', csshelper.CNeuReader(sqlite_connection))
     
        , ('cAgr', csshelper.CAgrReader(sqlite_connection))
    
        , ('cCon', csshelper.CConReader(sqlite_connection))
    
        , ('cOpn', csshelper.COpnReader(sqlite_connection)) 
]

In [3]:
aggregator = csstransformer.Aggregator([
        csstransformer.PartOfSpeech(),
        csstransformer.SentenceLength(),
        csstransformer.NumberOfWords(),
        csstransformer.NumberOfCommas(),
        csstransformer.NumberOfDots(),
        csstransformer.NumberOfSemicolons(),
        csstransformer.NumberOfColons(),
        csstransformer.LexicalDiversity(),
        csstransformer.AverageWordLength(),
        csstransformer.NumberOfFunctionalWords(),
        csstransformer.NumberOfPronouns(),
        csstransformer.NumberOfPropnames(),
])

data = csshelper.CExtReader(sqlite_connection).get_results()

# aggregator.transform(data.sentence)


In [4]:
grid_parameter_collection = {
    'classifierLinearSVC': { 'features__status__tf_idf_vect__ngram_range': ((1, 1), (1, 2)),
                           'features__derived_numeric__best': (5, 7,9, 10)},
    'classifierSVC': { 'features__status__tf_idf_vect__ngram_range': ((1, 1), (1, 2)),
                     'features__derived_numeric__best': (5,7,9,10)},
    'classifierNB': { 'features__status__tf_idf_vect__ngram_range': ((1, 1), (1, 2)),
                    'features__derived_numeric__best': (5,7,9, 10)},
    'classifierBNB': { 'features__status__tf_idf_vect__ngram_range': ((1, 1), (1, 2)),
                     'features__derived_numeric__best': (5,7, 9, 10)},
}
# foo =             ('derived_string', sklearn.pipeline.Pipeline([
#                 ('string_aggregator', csstransformer.Aggregator([
#                 csstransformer.PartOfSpeech()
#                 ])),
#               ('tf_idf_vect', sklearn.feature_extraction.text.TfidfVectorizer()),
#             ]))

base_pipeline = sklearn.pipeline.Pipeline([     
    ('features', sklearn.pipeline.FeatureUnion(
          transformer_list=[
            ('status', sklearn.pipeline.Pipeline([
              ('tf_idf_vect', sklearn.feature_extraction.text.TfidfVectorizer()),
            ])),
                  
            ('derived_string', sklearn.pipeline.Pipeline([
                ('part_of_speech', csstransformer.PartOfSpeech()),
                ('tf_idf_vect', sklearn.feature_extraction.text.TfidfVectorizer()),
            ])),

            
            ('derived_numeric', sklearn.pipeline.Pipeline([
              ('numeric_aggregator', csstransformer.Aggregator([
                    csstransformer.SentenceLength(),
                    csstransformer.NumberOfWords(),
                    csstransformer.NumberOfCommas(),
                    csstransformer.NumberOfDots(),
                    csstransformer.NumberOfSemicolons(),
                    csstransformer.NumberOfColons(),
                    csstransformer.LexicalDiversity(),
                    csstransformer.AverageWordLength(),
                    csstransformer.NumberOfFunctionalWords(),
                    csstransformer.NumberOfPronouns(),
                    csstransformer.NumberOfPropnames(),
                ])), 
              ('scaler', sklearn.preprocessing.RobustScaler()),
              ('best', sklearn.decomposition.TruncatedSVD(random_state=5152)),
            ])),   
          ],
    )),
])

classifier_collection = [
        ('classifierLinearSVC', ('clf', sklearn.svm.LinearSVC(random_state = 5152))),
#         ('classifierSVC2Variante', ('clf', sklearn.svm.SVC(cache_size=4096, random_state = 5152, 
#                                                   kernel = "linear", decision_function_shape = "ovr", 
#                                                 probability=True))),
        ('classifierSVC', ('clf', sklearn.svm.SVC(cache_size=4096, random_state = 5152, 
                                                  kernel = "rbf", decision_function_shape = "ovr", 
                                                probability=True))),
        ('classifierNB', ('nb', sklearn.naive_bayes.MultinomialNB())),
        ('classifierBNB', ('bnb', sklearn.naive_bayes.BernoulliNB()))
]

def create_full_pipelines():
    return [
        (class_name, sklearn.pipeline.Pipeline(base_pipeline.steps + [classifier]))
        for class_name, classifier in classifier_collection
    ]

pipeline_collection = create_full_pipelines()


In [5]:
for pipe_name, pipe in pipeline_collection:
    print()
    print(pipe_name)
#     print(pipe.steps)


classifierLinearSVC

classifierSVC

classifierNB

classifierBNB


In [6]:
def plot_roc_curve(test_class, y_score, name, trait):
    fpr, tpr, roc_auc = dict(), dict(), dict()
    fpr, tpr, _ = sklearn.metrics.roc_curve(test_class, y_score[:, 1], pos_label=1)
    roc_auc = sklearn.metrics.auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC chart with ' + name + " algorithm and on " + trait + " dataset")
    plt.legend(loc="lower right")
    plt.show()

In [11]:
for pipe_name, pipeline in pipeline_collection:
    for reader_description, reader in data_reader_collection:
        
        print("\nFOR: Pipeline: %s/%s" % (reader_description, pipe_name))

        data = reader.get_results()
        feature = data.iloc[:,0]
        label = data.iloc[:,1]
        
        grid_parameter = grid_parameter_collection[pipe_name]
        
        split = sklearn.cross_validation.train_test_split(feature, label, train_size=0.66, 
        stratify=label,random_state=5152)
        
        x_train, x_test, y_train, y_test = split
        
        grid_search = sklearn.grid_search.GridSearchCV(pipeline, param_grid=grid_parameter, cv=5, n_jobs=-1, verbose=0)

        grid_search.fit(x_train, y_train)
        y_pred_trait = grid_search.predict(x_test)
        
        
        if 'predict_proba' in dir(pipeline.steps[-1][-1]):
            y_score_trait = pipeline.predict_proba(x_test)
            plot_roc_curve(y_test, y_score_trait, "Pipeline", reader_description)

#         print(sklearn.metrics.classification_report(y_test, y_pred_trait, labels=[0, 1], target_names=["0", "1"]))
#         print(sklearn.metrics.confusion_matrix(y_test, y_pred_trait, labels=[0, 1]))
        print("F1: ", sklearn.metrics.f1_score(y_test, y_pred_trait, labels=[0, 1], average='binary'))
        print("Precision: ", sklearn.metrics.average_precision_score(y_test, y_pred_trait, average='micro'))
        print("Recall: ", sklearn.metrics.recall_score(y_test, y_pred_trait, labels=[0, 1], average='binary'))
        print("Accuracy score: ", sklearn.metrics.accuracy_score(y_test, y_pred_trait))
        
        print()
        
        print("GRID SEARCH:")
        print("Best score: %0.3f" % grid_search.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search.best_estimator_.get_params()
        for param_name in sorted(grid_parameter.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))


FOR: Pipeline: cExt/classifierLinearSVC
F1:  0.485703393061
Precision:  0.607503897134
Recall:  0.445143256464
Accuracy score:  0.599940688019

GRID SEARCH:
Best score: 0.597
Best parameters set:
	features__derived_numeric__best: TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=5152, tol=0.0)
	features__status__tf_idf_vect__ngram_range: (1, 2)

FOR: Pipeline: cNeu/classifierLinearSVC
F1:  0.385026737968
Precision:  0.535037309922
Recall:  0.313291139241
Accuracy score:  0.624851720047

GRID SEARCH:
Best score: 0.628
Best parameters set:
	features__derived_numeric__best: TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=5152, tol=0.0)
	features__status__tf_idf_vect__ngram_range: (1, 2)

FOR: Pipeline: cAgr/classifierLinearSVC
F1:  0.617574596222
Precision:  0.716117588649
Recall:  0.629815745394
Accuracy score:  0.585705812574

GRID SEARCH:
Best score: 0.575
Best parameters set:
	features__derived_numeric__best: Truncat

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [5]:
help(csstransformer)

Help on package csstransformer:

NAME
    csstransformer - # helpers

PACKAGE CONTENTS
    aggregator
    basetransformer
    lexicaldiversity
    nountransformer
    numberofdots
    partofspeech
    sentencelength
    smiley
    stemmedwords
    stemmer
    tagging
    tokenizer

FILE
    /home/dust/workspace/CaseSolvingSeminar/src/csstransformer/__init__.py


